<a href="https://colab.research.google.com/github/Az-Ks/Zindi-Weekendz/blob/master/5%C3%A9me_Place.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing  Data**

In [1]:
!pip install kaggle

In [0]:
#upload the credentials of the kaggle account
from google.colab import files
files.upload()

In [0]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions c!kaggle datasets download -d azerksouri/insurance-hackhange avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d azerksouri/insurance-hack

 48% 9.00M/18.9M [00:00<00:00, 12.0MB/s]
100% 18.9M/18.9M [00:00<00:00, 22.8MB/s]


In [10]:
!unzip "insurance-hack.zip"

Archive:  insurance-hack.zip
  inflating: client_data.csv         
  inflating: payment_history.csv     
  inflating: policy_data.csv         
  inflating: sample_sub (2).csv      
  inflating: train (7).csv           


In [0]:
import pandas as pd
import numpy as np
client_data=pd.read_csv('./client_data.csv')
policy_data=pd.read_csv('./policy_data.csv')
train=pd.read_csv('./train (7).csv')
payment_history=pd.read_csv('./payment_history.csv')
sample_sub=pd.read_csv('./sample_sub (2).csv')

## **policy_data**

In [35]:
date=['NP2_EFFECTDATE']
for i in date:    
    policy_data[i] = pd.to_datetime(policy_data[i])
    policy_data[i+'-year'] = policy_data[i].dt.year
    policy_data[i+'-month'] = policy_data[i].dt.month
    policy_data[i+'-day'] = policy_data[i].dt.day
policy_data.drop(['NP2_EFFECTDATE','NPH_LASTNAME'],axis=1,inplace=True)    
from sklearn.preprocessing import LabelEncoder
for col in policy_data.columns:
    if policy_data[col].dtypes == 'object':
      if col !='Policy ID':
        label_encoder =LabelEncoder() 
        policy_data[col]= label_encoder.fit_transform(policy_data[col]) 
policy_data        

,Policy ID,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,NP2_EFFECTDATE-year,NP2_EFFECTDATE-month,NP2_EFFECTDATE-day
0,PID_EPZDSP8,5,265.724174,2,222,NaN,1,NaN,182,5,155,2,2019,1,9
1,PID_6M6G9IB,1,2795.069380,1,111,213380.713197,4,609.054794,171,6,117,5,2018,1,8
2,PID_UL0F7LH,10,2492.759107,1,111,238857.872515,3,1339.461987,555,12,156,1,2017,1,8
3,PID_TRGUBTU,10,3982.538095,1,111,74968.903115,3,7870.961557,35,13,208,3,2018,1,4
4,PID_TODLPIB,10,1143.953733,3,555,238857.872515,1,NaN,226,1,155,2,2019,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282810,PID_7P0IXF8,10,2109.812579,3,555,74968.903115,1,NaN,35,13,117,5,2017,1,5
282811,PID_Q1QLPUQ,10,346.027849,2,222,45513.429325,1,NaN,363,13,117,5,2018,1,11
282812,PID_NEDS3A9,10,217.975890,3,333,45513.429325,1,NaN,217,1,155,2,2017,1,7
282813,PID_U8QKEJK,15,5233.874657,2,222,238857.872515,3,1075.892081,347,10,117,5,2017,1,6


In [36]:
print("len(set(payment_history['Policy ID']).difference(set(policy_data['Policy ID'])))",len(set(payment_history['Policy ID']).difference(set(policy_data['Policy ID']))))
print("len(set(policy_data['Policy ID']).difference(set(payment_history['Policy ID'])))",len(set(policy_data['Policy ID']).difference(set(payment_history['Policy ID']))))
print("len(set(train['Policy ID']).difference(set(payment_history['Policy ID'])))",len(set(train['Policy ID']).difference(set(payment_history['Policy ID']))))
print("len(set(payment_history['Policy ID']).difference(set(train['Policy ID'])))",len(set(payment_history['Policy ID']).difference(set(train['Policy ID']))))
print("len(set(policy_data['Policy ID']).difference(set(train['Policy ID'])))",len(set(policy_data['Policy ID']).difference(set(train['Policy ID']))))
print("len(set(train['Policy ID']).difference(set(policy_data['Policy ID'])))",len(set(train['Policy ID']).difference(set(policy_data['Policy ID']))))
print("len(set(sample_sub['Policy ID']).difference(set(train['Policy ID'])))",len(set(sample_sub['Policy ID']).difference(set(train['Policy ID']))))
print("len(set(train['Policy ID']).difference(set(sample_sub['Policy ID'])))",len(set(train['Policy ID']).difference(set(sample_sub['Policy ID']))))
print("len(set(sample_sub['Policy ID']).difference(set(payment_history['Policy ID'])))",len(set(sample_sub['Policy ID']).difference(set(payment_history['Policy ID']))))
print("len(set(payment_history['Policy ID']).difference(set(sample_sub['Policy ID'])))",len(set(payment_history['Policy ID']).difference(set(sample_sub['Policy ID']))))
print("len(set(sample_sub['Policy ID']).difference(set(policy_data['Policy ID'])))",len(set(sample_sub['Policy ID']).difference(set(policy_data['Policy ID']))))
print("len(set(policy_data['Policy ID']).difference(set(sample_sub['Policy ID'])))",len(set(policy_data['Policy ID']).difference(set(sample_sub['Policy ID']))))

len(set(payment_history['Policy ID']).difference(set(policy_data['Policy ID']))) 0
len(set(policy_data['Policy ID']).difference(set(payment_history['Policy ID']))) 19462
len(set(train['Policy ID']).difference(set(payment_history['Policy ID']))) 19462
len(set(payment_history['Policy ID']).difference(set(train['Policy ID']))) 0
len(set(policy_data['Policy ID']).difference(set(train['Policy ID']))) 0
len(set(train['Policy ID']).difference(set(policy_data['Policy ID']))) 0
len(set(sample_sub['Policy ID']).difference(set(train['Policy ID']))) 0
len(set(train['Policy ID']).difference(set(sample_sub['Policy ID']))) 7978
len(set(sample_sub['Policy ID']).difference(set(payment_history['Policy ID']))) 18450
len(set(payment_history['Policy ID']).difference(set(sample_sub['Policy ID']))) 6966
len(set(sample_sub['Policy ID']).difference(set(policy_data['Policy ID']))) 0
len(set(policy_data['Policy ID']).difference(set(sample_sub['Policy ID']))) 7978


In [0]:
train=pd.merge(train,policy_data,on='Policy ID',how='left')

In [38]:
aggs = {}
aggs['NLO_AMOUNT'] = ['max','min']
agg_trans = train.groupby(['Policy ID']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (train.groupby('Policy ID')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

agg_trans = pd.merge(df, agg_trans, on='Policy ID', how='left')
train = pd.merge(train,agg_trans, on='Policy ID', how='left')
train.head()

,Policy ID,Lapse,Lapse Year,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,NP2_EFFECTDATE-year,NP2_EFFECTDATE-month,NP2_EFFECTDATE-day,1transactions_count,NLO_AMOUNT_max,NLO_AMOUNT_min
0,PID_4928TWH,?,?,16,42911.077278,1,111,340667.737114,3,16702.717882,346,1,51,1,2017,1,8,1,16702.717882,16702.717882
1,PID_KBLLEGK,?,?,1,3561.268991,1,111,165053.885026,3,600.259636,271,11,48,3,2018,1,5,2,776.012205,600.259636
2,PID_KBLLEGK,?,?,1,3561.268991,1,111,165053.885026,4,776.012205,271,11,48,3,2018,1,5,2,776.012205,600.259636
3,PID_90F0QA3,?,?,1,6164.812836,1,111,252467.025723,4,1343.332956,204,9,155,2,2019,1,9,2,1343.332956,1039.092615
4,PID_90F0QA3,?,?,1,6164.812836,1,111,252467.025723,3,1039.092615,204,9,155,2,2019,1,9,2,1343.332956,1039.092615


In [39]:
train.drop_duplicates(subset=['Policy ID'],inplace=True) 
train.drop('NLO_AMOUNT', axis=1 , inplace=True )
train['extra amount'] = train['NLO_AMOUNT_max']-train['NLO_AMOUNT_min']
train

,Policy ID,Lapse,Lapse Year,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,NP2_EFFECTDATE-year,NP2_EFFECTDATE-month,NP2_EFFECTDATE-day,1transactions_count,NLO_AMOUNT_max,NLO_AMOUNT_min,extra amount
0,PID_4928TWH,?,?,16,42911.077278,1,111,340667.737114,3,346,1,51,1,2017,1,8,1,16702.717882,16702.717882,0.000000
1,PID_KBLLEGK,?,?,1,3561.268991,1,111,165053.885026,3,271,11,48,3,2018,1,5,2,776.012205,600.259636,175.752569
3,PID_90F0QA3,?,?,1,6164.812836,1,111,252467.025723,4,204,9,155,2,2019,1,9,2,1343.332956,1039.092615,304.240341
5,PID_18F3NHF,?,?,7,2278.189789,1,111,238857.872515,3,55,2,155,2,2019,1,12,1,1318.742708,1318.742708,0.000000
6,PID_SX4QUVO,?,?,15,1619.046308,1,111,145010.003453,3,347,10,155,2,2019,1,3,1,332.816358,332.816358,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282800,PID_C7EJ59O,1,2018,10,2915.304278,4,555,74968.903115,1,465,4,155,2,2017,1,6,7,1339.461987,1339.461987,0.000000
282807,PID_98JLBWI,?,?,12,614.308232,1,111,45513.429325,1,196,1,223,2,2019,1,6,2,1398.351185,1398.351185,0.000000
282809,PID_HM8H99C,?,?,1,3187.154301,1,111,86253.370871,3,168,6,211,2,2017,1,10,2,694.491386,537.201791,157.289595
282811,PID_KQH5U5I,?,?,1,2795.069380,1,111,109268.927230,4,264,13,211,2,2019,1,8,2,609.054794,471.115025,137.939769


In [0]:
train['Lapse']=train['Lapse'].str.replace('?','0')
train['Lapse Year']=train['Lapse Year'].str.replace('?','2019')
train['Lapse']=train['Lapse'].astype('int')
train['Lapse Year']=train['Lapse Year'].astype('int')

In [41]:
y_train=train['Lapse']
X_train=train.drop(['Lapse'],axis=1)
X_train

,Policy ID,Lapse Year,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,NP2_EFFECTDATE-year,NP2_EFFECTDATE-month,NP2_EFFECTDATE-day,1transactions_count,NLO_AMOUNT_max,NLO_AMOUNT_min,extra amount
0,PID_4928TWH,2019,16,42911.077278,1,111,340667.737114,3,346,1,51,1,2017,1,8,1,16702.717882,16702.717882,0.000000
1,PID_KBLLEGK,2019,1,3561.268991,1,111,165053.885026,3,271,11,48,3,2018,1,5,2,776.012205,600.259636,175.752569
3,PID_90F0QA3,2019,1,6164.812836,1,111,252467.025723,4,204,9,155,2,2019,1,9,2,1343.332956,1039.092615,304.240341
5,PID_18F3NHF,2019,7,2278.189789,1,111,238857.872515,3,55,2,155,2,2019,1,12,1,1318.742708,1318.742708,0.000000
6,PID_SX4QUVO,2019,15,1619.046308,1,111,145010.003453,3,347,10,155,2,2019,1,3,1,332.816358,332.816358,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282800,PID_C7EJ59O,2018,10,2915.304278,4,555,74968.903115,1,465,4,155,2,2017,1,6,7,1339.461987,1339.461987,0.000000
282807,PID_98JLBWI,2019,12,614.308232,1,111,45513.429325,1,196,1,223,2,2019,1,6,2,1398.351185,1398.351185,0.000000
282809,PID_HM8H99C,2019,1,3187.154301,1,111,86253.370871,3,168,6,211,2,2017,1,10,2,694.491386,537.201791,157.289595
282811,PID_KQH5U5I,2019,1,2795.069380,1,111,109268.927230,4,264,13,211,2,2019,1,8,2,609.054794,471.115025,137.939769


In [0]:
sample_sub=sample_sub[['Policy ID']]
test=pd.merge(sample_sub,train,on=['Policy ID'],how='left')
test.drop(['Lapse'],axis=1,inplace=True)

In [0]:
xx =X_train.copy()
tt =test.copy()

In [0]:
data = pd.concat([xx,tt])
numerical =['NPR_PREMIUM','NPR_SUMASSURED','NLO_AMOUNT_max','NLO_AMOUNT_min','extra amount']
data_num =data[numerical]
traintest  = data.drop(['Policy ID'] + numerical,axis=1 )

In [45]:
dummies = pd.get_dummies(traintest, columns=traintest.columns, drop_first=True, sparse=True)
train_ohe = dummies.iloc[:xx.shape[0], :]
test_ohe = dummies.iloc[xx.shape[0]:, :]

print(train_ohe.shape)
print(test_ohe.shape)

(51685, 989)
(43707, 989)


In [46]:
%%time
'''Covert dataframe to spare matrix'''
train_ohe = train_ohe.sparse.to_coo().tocsr()
test_ohe = test_ohe.sparse.to_coo().tocsr()
type(train_ohe)

CPU times: user 143 ms, sys: 17 µs, total: 143 ms
Wall time: 145 ms


In [0]:
target=y_train

## **MODELING**

In [0]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

## **KNN**   PRIVATE : 0.245033738959324 ,Public : 0.2464...

In [28]:
def runKNN(train_X, train_y, test, params):
    print('Train KNN')
    model = KNeighborsClassifier(**params)
    model.fit(train_X, train_y)
    
    print('Predict test')
    pred_test_y2 = model.predict_proba(test)[:, 1]
    return  pred_test_y2



KNN_params = { 'n_neighbors':  3300, 'leaf_size': 100}
pred_KNN = runKNN(train_ohe, target, test_ohe, KNN_params)

Train KNN
Predict test


In [0]:
sub_KNN =sample_sub.copy()
sub_KNN['Lapse']=pred_KNN
sub_KNN.to_csv('KNN.csv',index=False)

## **SVM** --**PUBLIC** :0.251444953490135 --Private : 0.251970058945692


In [30]:
def runSVM(train_X, train_y, test, params):
    print('Train SVM')
    model = SVC(**params)
    model.fit(train_X, train_y)
    
    print('Predict test')
    pred_test_y2 = model.predict_proba(test)[:, 1]
    return  pred_test_y2



SVM_params = { 'C' :  0.001 , 'kernel': 'linear' ,'random_state' :4 ,'probability' :True}
pred_svm = runSVM(train_ohe, target, test_ohe, SVM_params)

Train SVM
Predict test


In [0]:
sub_SVM =sample_sub.copy()
sub_SVM['Lapse']=pred_svm
sub_SVM.to_csv('SVM.csv',index=False)

## **ENSEMBLING**

In [0]:
ensemble =sample_sub.copy()
ensemble['Lapse'] = sub_KNN['Lapse'] * 0.8   + sub_SVM['Lapse'] * 0.2 
ensemble.to_csv('ensemble.csv',index=False)